In [1]:
#importing modules
import os
import codecs
import numpy as np
import string
import pandas as pd
import tarfile

# **Data Preprocessing**

In [2]:
#downloading and extracting the files on colab server
tar = tarfile.open("/Users/redap/Documents/GitHub/CS481/20_newsgroups.tar")
tar.extractall()
tar.close()

In [3]:
#making a list of all the file paths and their corresponding class
f_paths=[]
i=-1
path="20_newsgroups"
folderlist=os.listdir(path)
if ".DS_Store" in folderlist:
  folderlist.remove('.DS_Store')
for folder in folderlist:
  i+=1
  filelist=os.listdir(path+'/'+folder)
  for file in filelist:
    f_paths.append((path+'/'+folder+'/'+file,i))
len(f_paths)

19997

In [4]:
#splitting the list of paths into training and testing data
from sklearn import model_selection
x_train,x_test=model_selection.train_test_split(f_paths)
len(x_train),len(x_test)

(14997, 5000)

In [5]:
#Making the lists X_train and X_test containg only the paths of the files in training and testing data
#First making lists Y_train and Y_test containing the classes of the training and testing data
X_train=[]
X_test=[]
Y_train=[]
Y_test=[]
for i in range(len(x_train)):
  X_train.append(x_train[i][0])
  Y_train.append(x_train[i][1])
for i in range(len(x_test)):
  X_test.append(x_test[i][0])
  Y_test.append(x_test[i][1])
#Transforming Y_train and Y_test into 1 dimensional np arrays
Y_train=(np.array([Y_train])).reshape(-1)
Y_test=(np.array([Y_test])).reshape(-1)
#shape of Y_train and Y_test np arrays
Y_train.shape,Y_test.shape

((14997,), (5000,))

In [6]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop=set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /Users/redap/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
#adding all the above lists and including punctuations to stop words
stop_words=list(stop)+list(set(string.punctuation))
len(stop_words)

211

In [8]:
#making vocabulary from the files in X_train i.e. training data
vocab={}
count =0
for filename in X_train:
  count+=1
  f = open(filename,'r',errors='ignore')
  record=f.read()
  words=record.split()
  for word in words:
    if len(word)>2:
      if word.lower() not in stop_words:
        if word.lower() in vocab:
          vocab[word.lower()]+=1
        else:
          vocab[word.lower()]=1
  f.close()

In [9]:
#length of the vocabulary
len(vocab)

353458

In [10]:
#sorting the vocabulary on the basis of the frequency of the word
#making the sorted vocabulary
import operator
sorted_vocab = sorted(vocab.items(), key= operator.itemgetter(1), reverse= True)   # sort the vocab based on frequency

In [11]:
#making the list feature_names containg the words with the frequency of the top 2000 words
feature_names = []
for i in range(len(sorted_vocab)):
    if(sorted_vocab[2000][1] <= sorted_vocab[i][1]):
        feature_names.append(sorted_vocab[i][0])

In [12]:
#length of the feature_names i.e. number of our features
print(len(feature_names))

2012


In [13]:
#making dataframes df_train and df_test with columns having the feature names i.e. the words
df_train=pd.DataFrame(columns=feature_names)
df_test=pd.DataFrame(columns=feature_names)

In [14]:
count_train,count_test=0,0

#transforming each file in X_train into a row in the dataframe df_train having columns as feature names and values as the frequency of that feature name i.e that word
for filename in X_train:
  count_train+=1
  #adding a row of zeros for each file
  df_train.loc[len(df_train)]=np.zeros(len(feature_names))
  f = open(filename,'r',errors='ignore')
  record=f.read()
  words=record.split()
  #parsing through all the words of the file
  for word in words:
    if word.lower() in df_train.columns:
      df_train[word.lower()][len(df_train)-1]+=1 #if the word is in the column names then adding 1 to the frequency of that word in the row
  f.close()
  
#transforming each file in X_test into a row in the dataframe df_test having columns as feature names and values as the frequency of that feature name i.e that word  
for filename in X_test:
  count_test+=1
  #adding a row of zeros for each file
  df_test.loc[len(df_test)]=np.zeros(len(feature_names))
  f = open(filename,'r',errors='ignore')
  record=f.read()
  words=record.split()
  #parsing through all the words of the file
  for word in words:
    if word.lower() in df_test.columns:
      df_test[word.lower()][len(df_test)-1]+=1 #if the word is in the column names then adding 1 to the frequency of that word in the row
  f.close()
  
#printing the number files tranformed in training and testing data
print(count_train,count_test)

In [ ]:
#putting the values of the datafames into X_train and X_test
X_train=df_train.values
X_test=df_test.values

# **Using the inbuilt Multinomial Naive Bayes classifier from sklearn**

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report,confusion_matrix
clf=MultinomialNB()
#fitting the classifier on training data
clf.fit(X_train,Y_train)
#prediciting the classes of the testing data
Y_pred=clf.predict(X_test)
#classification report
print(classification_report(Y_test,Y_pred))
#testing score
print("Testing: ",clf.score(X_test,Y_test))

              precision    recall  f1-score   support

           0       0.94      0.77      0.85       245
           1       0.78      0.91      0.84       255
           2       0.93      0.91      0.92       233
           3       0.71      0.80      0.75       263
           4       0.94      0.95      0.95       253
           5       0.83      0.80      0.81       244
           6       0.97      0.97      0.97       280
           7       0.95      0.84      0.90       269
           8       0.90      0.87      0.88       251
           9       0.67      0.60      0.63       246
          10       0.84      0.92      0.88       267
          11       0.89      0.89      0.89       236
          12       0.82      0.77      0.79       217
          13       0.88      0.83      0.85       273
          14       0.70      0.87      0.78       220
          15       0.74      0.87      0.80       239
          16       0.92      0.80      0.85       242
          17       0.95    

# **Save Model**

In [ ]:
from joblib import dump

model_dir = '/Users/redap/Documents/GitHub/CS481' 
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

model_path = os.path.join(model_dir, 'multinomial_nb_model.joblib')
dump(clf, model_path)

print(os.path.exists(model_path)) 


True
